In [45]:
import pandas as pd    # I got this block of code from colleagues and online tools....through hours of trial and error 

df = pd.read_csv("DW3_set_2.csv")

df["Dxcode"] = df["Dxcode"].astype(str).str.strip().str.upper()
df["Dxdate"] = pd.to_datetime(df["Dxdate"], errors="coerce")

diabetes_codes = ("E08", "E09", "E10", "E11", "E13")
covid_codes = {"U07.1", "J12.82"}

diab_dx = df[df["Dxcode"].str.startswith(diabetes_prefixes)].copy()    # Diabetes -- df defined 
covid_dx = df[df["Dxcode"].isin(covid_codes)].copy()                 # COVID

DIAB_PATIENTS  = set(diab_dx["PatientID"].dropna())        # Patient data 
COVID_PATIENTS = set(covid_dx["PatientID"].dropna())

both = COVID_PATIENTS & DIAB_PATIENTS
covid_only = COVID_PATIENTS - DIAB_PATIENTS    # setting paramenters 
diab_only  = DIAB_PATIENTS - COVID_PATIENTS
either = COVID_PATIENTS | DIAB_PATIENTS

print("COVID patients:", len(COVID_PATIENTS))
print("Diabetes patients:", len(DIAB_PATIENTS))
print("COVID ∩ Diabetes:", len(both))
print("COVID only:", len(covid_only))
print("Diabetes only:", len(diab_only))
print("COVID ∪ Diabetes:", len(either))

first_covid = covid_dx.groupby("PatientID")["Dxdate"].min()    # diabetes AFTER covid 
first_diab  = diab_dx.groupby("PatientID")["Dxdate"].min()

aligned = pd.DataFrame({
    "covid_date": first_covid,
    "diab_date": first_diab
}).dropna()

new_diab_after_covid = aligned[aligned["diab_date"] > aligned["covid_date"]]

print("New diabetes AFTER COVID:", len(new_diab_after_covid))

COVID patients: 1997
Diabetes patients: 6735
COVID ∩ Diabetes: 224
COVID only: 1773
Diabetes only: 6511
COVID ∪ Diabetes: 8508
New diabetes AFTER COVID: 94


In [46]:
df.head(10)

,RowID,PatientID,Dxcode,Dxdate
0,0,4725559,E03.9,2020-09-20
1,1,7637008,R53.1,2019-10-27
2,2,5068050,I48.1,2020-11-22
3,3,1342235,E11.9,2022-01-27
4,4,4520230,N39.0,2020-05-17
5,5,972635,E03.9,2022-01-05
6,6,1403893,N/A*,2020-12-14
7,7,5238223,F32.9,2019-10-23
8,8,3690912,M54.5,2021-09-18
9,9,1625668,M54.5,2022-02-09


In [48]:
df.shape

(40606, 4)

In [49]:
df.head()
df.info()
covid_dx.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40606 entries, 0 to 40605
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   RowID      40606 non-null  int64         
 1   PatientID  40606 non-null  int64         
 2   Dxcode     40606 non-null  object        
 3   Dxdate     40606 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.2+ MB


,RowID,PatientID,Dxcode,Dxdate
31,31,5692899,J12.82,2021-11-04
44,44,665176,J12.82,2022-03-11
52,52,3651321,U07.1,2019-10-21
76,76,1282153,J12.82,2022-01-25
91,91,5112391,J12.82,2020-06-04


In [50]:
df.head()
df.info()
diab_dx.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40606 entries, 0 to 40605
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   RowID      40606 non-null  int64         
 1   PatientID  40606 non-null  int64         
 2   Dxcode     40606 non-null  object        
 3   Dxdate     40606 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.2+ MB


,RowID,PatientID,Dxcode,Dxdate
3,3,1342235,E11.9,2022-01-27
13,13,1841379,E10,2021-07-16
15,15,6120921,E09,2020-03-06
19,19,3465224,E09,2021-12-06
29,29,1358532,E10,2019-10-09


In [51]:
# Question: #1a) List of patients with diabetes 

diabetes_patients = diab_dx["PatientID"].dropna().unique()

diabetes_patients     # this output is showing a NumPy array of PatientIDs

array([1342235, 1841379, 6120921, ..., 7160137, 6607308, 1340228],
      shape=(6735,))

In [52]:
# Q: 1a    total of 6,735 individual patients 

In [53]:
pd.DataFrame(diabetes_patients, columns=["PatientID"]).head()


,PatientID
0,1342235
1,1841379
2,6120921
3,3465224
4,1358532


In [54]:
# Q: 1b    This is a count of how many patients have at least one diabetes code

num_diabetes_patients = len(diabetes_patients)     

num_diabetes_patients

6735

In [55]:
# Q: 1c   Are there patients with more than one diabetes record?

diabetes_counts = (
    diab_dx
    .groupby("PatientID")
    .size()
    .reset_index(name="diabetes_dx_count")
)

patients_multiple_diab = diabetes_counts[diabetes_counts["diabetes_dx_count"] > 1]

patients_multiple_diab.head()

,PatientID,diabetes_dx_count
3,104363,2
9,117296,2
31,145957,2
52,178480,2
54,179518,2


In [57]:
# 1c  count how many patients exactly 

len(patients_multiple_diab)

489

In [58]:
# 2a  Here is a list of patients with at least one COVID diagnosis. 

covid_patients = covid_dx["PatientID"].dropna().unique()

covid_patients

# Patients with COVID were id by filtering the diagnosis data using ICD-10 codes ( U07.1 and J12.82 ). Patient IDs were extracted to 
# form the COVID patient data set 

array([5692899,  665176, 3651321, ..., 5331556, 5080120, 2398290],
      shape=(1997,))

In [59]:
pd.DataFrame(covid_patients, columns=["PatientID"]).head()

,PatientID
0,5692899
1,665176
2,3651321
3,1282153
4,5112391


In [60]:
# 2b Here is the count of patients with COVID
num_covid_patients = len(covid_patients)

num_covid_patients

1997

In [61]:
# 2c Patients with more than one COVID diagnosis record

covid_counts = (
    covid_dx
    .groupby("PatientID")
    .size()
    .reset_index(name="covid_dx_count")
)

patients_multiple_covid = covid_counts[covid_counts["covid_dx_count"] > 1]

patients_multiple_covid.head()

,PatientID,covid_dx_count
41,242212,2
59,287739,2
76,344867,2
205,856514,2
249,1060121,2


In [62]:
len(patients_multiple_covid)   # the exact number of patients with more than 1 COVID diagnosis 

29

In [63]:
# Question 3: Patients with Diabetes OR Covid -- Union method 

diabetes_patients = set(diab_dx["PatientID"].dropna())
covid_patients    = set(covid_dx["PatientID"].dropna())

diabetes_or_covid = diabetes_patients | covid_patients

diabetes_or_covid

{6012934,
 1736711,
 7012360,
 1376265,
 7290887,
 4079623,
 3981318,
 4390925,
 1900560,
 4456467,
 6127641,
 5390361,
 1769501,
 3817505,
 5685284,
 7340071,
 4751400,
 5521449,
 6750250,
 2277420,
 147506,
 2048051,
 5046324,
 7700533,
 6701106,
 3113015,
 1982519,
 2932790,
 3866682,
 3637307,
 7618620,
 5914682,
 3768382,
 5161024,
 2359362,
 6127682,
 540740,
 5390405,
 2179145,
 6471754,
 5390410,
 6062156,
 7192653,
 4063310,
 2883663,
 4849744,
 2031697,
 4653138,
 4030547,
 6946898,
 7520334,
 1097807,
 4767823,
 7225430,
 6471767,
 4210777,
 1425497,
 4341852,
 7225429,
 458846,
 7307359,
 6815836,
 2818145,
 4587614,
 5554274,
 540772,
 2998371,
 901222,
 7684202,
 6635635,
 1654900,
 7536757,
 5079162,
 1310845,
 1245312,
 6373505,
 1851521,
 7667843,
 7520387,
 6226053,
 5341314,
 5734535,
 7667848,
 1654922,
 1622157,
 3539086,
 3358868,
 3063958,
 1228953,
 4505754,
 6619292,
 4341920,
 1458336,
 7700642,
 4538532,
 6242476,
 5980333,
 3260588,
 6013104,
 2670769,
 6422

In [65]:
#3 conti.  How many patients are a part of the union 

len(diabetes_or_covid)

8508

In [68]:
# 3 Summary 
# A union operation was used to find all patients who had at least one diagnosis of diabetes or COVID. 

In [69]:
# Question: 4 Patients with Diabetes AND Covid --Intersection method. 

diabetes_patients = set(diab_dx["PatientID"].dropna())
covid_patients    = set(covid_dx["PatientID"].dropna())

diabetes_and_covid = diabetes_patients & covid_patients

diabetes_and_covid

{138575,
 175474,
 222248,
 227866,
 266223,
 269196,
 270723,
 311557,
 458846,
 482256,
 597141,
 606183,
 608265,
 620359,
 620623,
 626112,
 628441,
 663704,
 665855,
 700390,
 734644,
 765097,
 788972,
 818167,
 846756,
 848666,
 851267,
 942974,
 956890,
 992024,
 1032811,
 1053804,
 1066366,
 1080874,
 1099944,
 1113301,
 1165591,
 1201084,
 1220613,
 1346682,
 1376904,
 1408551,
 1415399,
 1428367,
 1434515,
 1441531,
 1448346,
 1464780,
 1500368,
 1596246,
 1603639,
 1610879,
 1700963,
 1759825,
 1781822,
 1789370,
 1811725,
 1856152,
 1931212,
 1980087,
 1998501,
 2028809,
 2217126,
 2221314,
 2290734,
 2299370,
 2299669,
 2301586,
 2316131,
 2320798,
 2348772,
 2349833,
 2356742,
 2369658,
 2403848,
 2417040,
 2465811,
 2481330,
 2483513,
 2600479,
 2629565,
 2646629,
 2683802,
 2694265,
 2708536,
 2719049,
 2764472,
 2766025,
 2769295,
 2805177,
 2827584,
 2831596,
 2892229,
 2986113,
 3058621,
 3078792,
 3091854,
 3122196,
 3131294,
 3159239,
 3178905,
 3256234,
 3262537,


In [71]:
# Lets count how many total 
len(diabetes_and_covid)

224

In [72]:
# 4 summary 
# This was defined using an intersection of the diabetes and Covid patient sets.

In [76]:
# Question #5
first_covid = covid_dx.groupby("PatientID")["Dxdate"].min()   # first COVID dx per pt

first_diab = diab_dx.groupby("PatientID")["Dxdate"].min()  # first diabeteds dx per pt

aligned = pd.DataFrame({      # align pt's with BOTH 
    "covid_date": first_covid,
    "diab_date": first_diab
}).dropna()

diabetes_after_covid = aligned[aligned["diab_date"] > aligned["covid_date"]]  # diabetes happening AFTER covid dx

diabetes_after_covid

,covid_date,diab_date
PatientID,,
138575,2021-02-17,2021-08-02
175474,2019-06-22,2020-03-20
222248,2019-09-12,2020-10-06
227866,2019-09-09,2020-10-12
482256,2019-11-19,2022-05-30
...,...,...
7588327,2020-12-18,2021-04-24
7588462,2020-01-02,2020-07-19
7609020,2021-03-29,2022-02-13


In [77]:
# how many total after COVID dx  -- you can see all these stats in cell line #1 with original code and df design

len(diabetes_after_covid)

94